In [17]:
import pandas as pd
import numpy as np
from datetime import datetime
import random

# ================== PARÂMETROS DO GERADOR (ajuste à vontade) ==================
N = 1000000
SEED = 42

# Força dos efeitos (quanto maior, mais sinal para o Prophet)
FORCA_SAZONAL_MES   = 1.50   # pico em nov/dez (ex.: 1.5 = +50%)
FORCA_SAZONAL_JUN   = 1.15   # leve alta em junho
FORCA_DIA_SEMANA    = 0.10   # ±10% entre dias úteis e fim de semana
FORCA_COMUNIDADE    = 0.12   # diferenças entre regiões (até ~±12%)
BASE_LAMBDA         = 0.60   # intensidade média por pedido (quanto maior, mais quantidade)
FORCA_LONG_TAIL     = 0.90   # expoente da lei de potência (0.7…1.2)
CICLO_CENTRO_ANO    = 2023.3 # centro do ciclo de vida
CICLO_INCLINACAO    = 3.0    # inclinação do ciclo de vida
CICLO_ESCALA_MIN    = 0.7    # faixa ~[0.7, 1.3]
CICLO_ESCALA_MAX    = 1.3

np.random.seed(SEED)
random.seed(SEED)

# ================== DADOS ESTÁTICOS ==================
estados = [
    'Andalucía', 'Cataluña', 'Madrid', 'Valencia', 'Galicia',
    'Castilla y León', 'País Vasco', 'Canarias', 'Aragón', 'Murcia'
]

# ciudades (para manter compatibilidade com seu mapeamento)
almacenes = [
    'Sevilla', 'Barcelona', 'Madrid', 'Valencia', 'Santiago de Compostela',
    'Valladolid', 'Bilbao', 'Las Palmas', 'Zaragoza', 'Murcia'
]

mapa_almacen = dict(zip(estados, almacenes))

categorias = ['Electrónica', 'Ropa', 'Belleza', 'Hogar', 'Deporte', 'Juguetes',
              'Automotriz', 'Libros', 'Mascotas', 'Informática']

productos = [f'P{i:04d}' for i in range(1, 151)]
cat_map = {p: random.choice(categorias) for p in productos}
nombre_map = {p: f'producto_{i}' for i, p in enumerate(productos, start=1)}

# ================== PESOS DE POPULARIDADE (LONG TAIL) ==================
ranks = np.arange(1, len(productos) + 1)
pop_weights = 1 / (ranks ** FORCA_LONG_TAIL)
pop_weights = pop_weights / pop_weights.sum()
prod_weights = dict(zip(productos, pop_weights))

# ================== FATORES DE QUANTIDADE ==================
# Mensal (nov/dez pico; jun leve alta)
fator_mensal_qtd = {m: 1.0 for m in range(1, 13)}
for m in (11, 12): fator_mensal_qtd[m] = FORCA_SAZONAL_MES
fator_mensal_qtd[6] = FORCA_SAZONAL_JUN

# Dia da semana
# 0=seg … 6=dom. Aumenta dias úteis, reduz fim de semana
efeito_dow = {
    0: 1 + FORCA_DIA_SEMANA, 1: 1 + FORCA_DIA_SEMANA, 2: 1 + FORCA_DIA_SEMANA,
    3: 1 + FORCA_DIA_SEMANA, 4: 1 + FORCA_DIA_SEMANA,
    5: 1 - FORCA_DIA_SEMANA/2, 6: 1 - FORCA_DIA_SEMANA
}

# Comunidade (variação suave em torno de 1.0)
efeito_estado = {
    'Andalucía': 1 + FORCA_COMUNIDADE,
    'Cataluña': 1 + FORCA_COMUNIDADE*0.8,
    'Madrid':   1 + FORCA_COMUNIDADE*1.1,
    'Valencia': 1 + FORCA_COMUNIDADE*0.2,
    'Galicia':  1 - FORCA_COMUNIDADE*0.3,
    'Castilla y León': 1 - FORCA_COMUNIDADE*0.5,
    'País Vasco': 1.00,
    'Canarias':  1 - FORCA_COMUNIDADE*0.7,
    'Aragón':    1 - FORCA_COMUNIDADE*0.5,
    'Murcia':    1 - FORCA_COMUNIDADE*0.25,
}

# Efeito por categoria (leve)
cat_eff = {
    'Electrónica': 1.05, 'Ropa': 1.00, 'Belleza': 0.98, 'Hogar': 1.00, 'Deporte': 1.02,
    'Juguetes': 1.03, 'Automotriz': 0.97, 'Libros': 0.96, 'Mascotas': 1.00, 'Informática': 1.04
}

# ================== CICLO DE VIDA (função logística) ==================
def ciclo_vida_factor(ts: pd.Timestamp) -> float:
    ano = ts.year + (ts.dayofyear / 365)
    x = CICLO_INCLINACAO * (ano - CICLO_CENTRO_ANO)
    # mapeia logística para [CICLO_ESCALA_MIN, CICLO_ESCALA_MAX]
    sig = 1 / (1 + np.exp(-x))
    return CICLO_ESCALA_MIN + (CICLO_ESCALA_MAX - CICLO_ESCALA_MIN) * sig

# ================== GERADOR ==================
def generar_datos(n=N):
    # Linha do tempo por hora
    fechas = pd.date_range(start='2022-01-01', end='2024-12-31', freq='H')
    fechas_sample = np.random.choice(fechas, n)

    # Produto com probabilidade (popularidade)
    productos_sample = np.random.choice(productos, n, p=[prod_weights[p] for p in productos])

    estados_sample = np.random.choice(
        estados, n, p=[0.18, 0.15, 0.14, 0.10, 0.08, 0.08, 0.07, 0.06, 0.07, 0.07]
    )
    categorias_sample = [cat_map[p] for p in productos_sample]
    nombres_sample = [nombre_map[p] for p in productos_sample]
    almacen_sample = [mapa_almacen[e] for e in estados_sample]

    # ===== Intensidade (lambda) para QUANTIDADE =====
    meses = pd.to_datetime(fechas_sample).month
    dows  = pd.to_datetime(fechas_sample).dayofweek

    lam = (BASE_LAMBDA
           * np.array([fator_mensal_qtd[m] for m in meses])                 # sazonalidade mensal (qtd)
           * np.array([efeito_dow[d] for d in dows])                         # dia da semana
           * np.array([efeito_estado[e] for e in estados_sample])            # efeito UF
           * np.array([ciclo_vida_factor(ts) for ts in pd.to_datetime(fechas_sample)])  # ciclo de vida
           * np.array([cat_eff[c] for c in categorias_sample]))              # efeito categoria

    # reforço por popularidade do SKU (sem explodir valores)
    # normaliza pelo SKU mais popular para virar multiplicador ~[1..~2.5]
    best_weight = max(prod_weights.values())
    lam *= (1 + 1.5 * np.array([prod_weights[p] / best_weight for p in productos_sample]))

    # Quantidade ~ Poisson(lam), truncada a [0..5], com "pedido mínimo" empurrando parte dos zeros p/ 1
    cantidades = np.random.poisson(lam).clip(0, 5)
    mask_zero = (cantidades == 0)
    if mask_zero.any():
        cantidades[mask_zero] = np.random.choice([0, 1], mask_zero.sum(), p=[0.25, 0.75])

    # ===== Preço (mantém sua sazonalidade em nov/dez) =====
    precio_base = {
        'Electrónica': (800, 2500),
        'Ropa': (80, 300),
        'Belleza': (30, 120),
        'Hogar': (100, 600),
        'Deporte': (150, 800),
        'Juguetes': (40, 200),
        'Automotriz': (200, 1500),
        'Libros': (30, 120),
        'Mascotas': (50, 250),
        'Informática': (300, 2000)
    }
    precios = np.array([np.random.uniform(*precio_base[c]) for c in categorias_sample])
    factor_estacional_precio = np.where(np.isin(meses, [11, 12]), np.random.uniform(1.3, 1.8, n), 1.0)
    precios = np.round(precios * factor_estacional_precio, 2)

    # Tempo de entrega (igual ao seu)
    entrega = np.random.normal(3.5, 1.2, n) + np.random.choice(range(1, 5), n)
    entrega = np.clip(entrega, 1, 10)

    df = pd.DataFrame({
        'id_pedido': np.arange(1, n + 1),
        'fecha_pedido': fechas_sample,
        'comunidad_autonoma': estados_sample,
        'id_producto': productos_sample,
        'nombre_producto': nombres_sample,
        'categoria': categorias_sample,
        'precio': precios,
        'cantidad': cantidades,
        'tiempo_entrega_dias': np.round(entrega, 1),
        'ciudad': almacen_sample
    })
    return df

# ================== EXECUÇÃO ==================
print("Generando base de datos de ventas en España (com sazonalidade de QUANTIDADE)…")
df = generar_datos()

output_path = r"X:\git\previsao_itens_vendas\ecommerce_sintetico_espana.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✅ Base guardada correctamente en:\n{output_path}")


Generando base de datos de ventas en España (com sazonalidade de QUANTIDADE)…
✅ Base guardada correctamente en:
X:\git\previsao_itens_vendas\ecommerce_sintetico_espana.csv
